In [5]:
import pandas as pd
import datetime
from sklearn import datasets
import logging


In [8]:
from evidently.ui.workspace.cloud import CloudWorkspace

from evidently.report import Report
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import DataDriftPreset
from evidently.metrics import *
from evidently.test_suite import TestSuite
from evidently.tests import *
from evidently.test_preset import DataDriftTestPreset
from evidently.tests.base_test import TestResult, TestStatus
from src.DataOps.feature_engg import feature_engg_class
from src.ModelOps.model_fit import ModelFit
import wandb

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
#connecting to evidently cloud workspace using API token
ws = CloudWorkspace(
token=os.getenv('evi_api'),
url="https://app.evidently.cloud")

# #creating a project in the workspace
__project_name__="Test2 Project"
__team_id__="74fdd884-9679-4693-819b-a6695e652e25"

project = ws.create_project(__project_name__,team_id=__team_id__)
project.description = "My project description"
project.save()

Project(id=UUID('009a6dda-dba9-4f8c-8e41-67418d1f9dfc'), name='Test2 Project', description='My project description', dashboard=DashboardConfig(name='Test2 Project', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=UUID('74fdd884-9679-4693-819b-a6695e652e25'), date_from=None, date_to=None, created_at=datetime.datetime(2024, 6, 18, 20, 54, 45, 71256))

In [4]:
stock_data = feature_engg_class()
data=stock_data.request_stock_price_hist('AAPL')
stock_data.data=data
train,test=stock_data.split(use_prophet=True)

INFO:root:Feature Engineering class initialized
INFO:root:Retrieving stock price data from Alpha Vantage (This may take a while)...
INFO:root:Data has been successfully downloaded...
INFO:root:Storing the retrieved data into a dataframe...
100%|██████████| 1285/1285 [00:01<00:00, 695.75it/s]
INFO:root:Sorting the index and changing column data types...
INFO:root:Data loaded successfully
INFO:root:Data shape: (1285, 7)
INFO:root:Earliest Date:2024-06-18 00:00:00,Latest Date:2024-06-18 00:00:00
INFO:root:Data prepared for Prophet
INFO:root:Data split into train and test successfully
INFO:root:Train shape: 899
INFO:root:Test shape: 386


In [7]:
import yaml
with open('conf/mlops.yaml', 'r') as file:
            config = yaml.safe_load(file)
model_name = config['model_name']
project_name=config['project_name']
model_type=config['model_type']
wandb_entity=config['wandb_entity']
print(model_name,project_name,model_type,wandb_entity)

stock_price_prediction ml-ops-template classification sakshamgulati123


In [12]:
from prophet.serialize import  model_from_json

run = wandb.init(project=model_name,job_type=model_type)
artifact = run.use_artifact(f'{wandb_entity}/model-registry/{model_name}:latest', type='model')
artifact_dir = artifact.download()
logging.info(f"Artifact downloaded at: {artifact_dir}")
logging.info("Model artifact downloaded")
#load  pickle file
file_path = os.path.join(artifact_dir, 'serialized_model.json')  # specify the correct file path
with open(file_path, 'r') as fin:
    model = model_from_json(fin.read())  # Load model
logging.info("Model loaded from the registry")
y_pred = model.predict(test)


wandb:   1 of 1 files downloaded.  
INFO:root:Artifact downloaded at: /Users/sakshamgulati/MLOps_Template/artifacts/run-6g0znzbn-serialized_model.json:v0
INFO:root:Model artifact downloaded
INFO:root:Model loaded from the registry


In [14]:
ref_dataset = run.use_artifact(f'{wandb_entity}/{model_name}/reference-dataset:latest', type='dataset')
ref_dataset_dir = ref_dataset.download()
logging.info(f"Artifact downloaded at: {ref_dataset_dir}")
logging.info("Reference artifact downloaded")

wandb:   1 of 1 files downloaded.  
INFO:root:Artifact downloaded at: /Users/sakshamgulati/MLOps_Template/artifacts/reference-dataset:v3
INFO:root:Reference artifact downloaded


wandb: Network error (ConnectionError), entering retry loop.
